In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Условие
a, b = 0, 1
real_res = -0.1978168271761323678264082
def func(x):
    return (x**2 - 1) * 10**(-2 * x)

# Разбиение отрезка [a, b]
N = 10
h = (b - a) / N
split = [h * i for i in range(N + 1)]
print(f"Разбиение:\n{split}")

Разбиение:
[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0]


In [3]:
# Простейшие правые прямоугольники
def r_triangle(f, a, b):
    return (b - a) * f(b)

# Результат по простейшей формуле
res = r_triangle(func, a, b)
print(res)

0.0


In [4]:
# Составные правые прямоугольники по правилу Рунге
def r_triangle_comp(f, a, b, epsilon):
    h1 = 0.1
    Ih1 = 0
    N = int((b - a) / h1)
    for i in range(N):
        Ih1 += f(a + (i + 1) * h1)
    Ih1 *= h1
    while True:
        h2 = 0.5 * h1
        N = int((b - a) / h2)
        Ih2 = 0
        for i in range(N):
            Ih2 += f(a + (i + 1) * h2)
        Ih2 *= h2
        if abs((Ih2 - Ih1) / (1 - h2 / h1)) <= epsilon:
            return Ih2, h2
        h1, Ih1 = h2, Ih2


eps = 1e-5
res_r_tr_comp = r_triangle_comp(func, a, b, eps)
print(f"Значение интеграла: {res_r_tr_comp[0]}\nШаг разбиения: {res_r_tr_comp[1]}")
print(f"Невязка с реальным решением: {abs(res_r_tr_comp[0]) - abs(real_res)}")

Значение интеграла: -0.19781377543255163
Шаг разбиения: 6.103515625e-06
Невязка с реальным решением: -3.051743580728683e-06


In [5]:
# Оцениваем разбиения
def func_der(x):
    return 2 * x * pow(10, -2 * x) + (x**2 - 1) * math.log(10) * pow(10, -2 * x) * (- 2)

def func_der_2(x):
    return (2  - 8 * math.log(10) * x  + 4 * math.log(100) * x**2 - 4 * math.log(100)) * pow(10, -2 * x)

In [6]:
# Максимум второй производной
x = np.arange(a, b, 0.00001)
M2 = max([abs(func_der_2(i)) for i in x])
# M2 = 19.2076
print(F"M2: {M2}")
N_mid = math.ceil(math.sqrt((b - a)**3 / (24 * eps) * M2))
print(f"Число разбиений для средних прямоугольников: {N_mid}")
h_mid = (b - a) / N_mid
print(f"Шаг для средних прямоугольников: {h_mid}")

M2: 16.420680743952367
Число разбиений для средних прямоугольников: 262
Шаг для средних прямоугольников: 0.003816793893129771


In [7]:
M4 = 195.27086
N_simp = math.ceil(pow((b - a)**5 / (2880 * eps) * M4, 1 / 4))
print(f"Число разбиений для Симпсона: {N_simp}")
h_simp = (b - a) / N_simp
print(f"Шаг для Симпсона: {h_simp}")

Число разбиений для Симпсона: 10
Шаг для Симпсона: 0.1


In [8]:
# Средние прямоугольники
def m_triangle_comp(f, a, b, h):
    N = int((b - a) / h)
    sum = 0
    for i in range(N):
        sum += f(a + (i + 0.5) * h)
    return sum * h

res_mid_tr_comp = m_triangle_comp(func, a, b, h_mid)
print(f"Значение интеграла: {res_mid_tr_comp}")
print(f"Невязка с реальным решением: {abs(res_mid_tr_comp) - abs(real_res)}")

Значение интеграла: -0.19781404401175506
Невязка с реальным решением: -2.783164377295755e-06


In [9]:
# Симпсон
def simp_comp(f, a, b, h):
    N = int((b - a) / h)
    split = [h * i for i in range(N + 1)]
    temp1 = sum([f(split[i]) for i in range(0, N - 1, 2)])
    temp2 = sum([f(split[i]) for i in range(1, N, 2)])
    return h / 3 * (split[0] + split[N] + 2 * temp1 + 4 * temp2)


res_simp_comp = simp_comp(func, a, b, h_simp)
print(f"Значение интеграла: {res_simp_comp}")
print(f"Невязка с реальным решением: {abs(res_simp_comp) - abs(real_res)}")

Значение интеграла: -0.1978550914154395
Невязка с реальным решением: 3.8264239307139736e-05


In [10]:
# НАСТ Гаусса при n = 4
# Корни n + 1 многочлена Лежандра
roots = [0, 1 / 3 * math.sqrt(5 - 2 * math.sqrt(10 / 7)), - 1 / 3 * math.sqrt(5 - 2 * math.sqrt(10 / 7)),
         1 / 3 * math.sqrt(5 + 2 * math.sqrt(10 / 7)), - 1 / 3 * math.sqrt(5 + 2 * math.sqrt(10 / 7))]
print(roots)

[0, 0.538469310105683, -0.538469310105683, 0.906179845938664, -0.906179845938664]


In [11]:
# # Преобразуем корни для отрезка [a, b]
# for i in range(len(roots)):
#     roots[i] = (b - a) / 2 * roots[i] + (a + b) / 2
# print(roots)

In [12]:
# Коэффициенты Ak
def p_der_5(x):
    return 15 / 8 * (21 * x**4 - 14 * x**2 + 1)

A = []
for root in roots:
    A.append(2 / ((1 - root**2) * p_der_5(root)**2))
print(A)

[0.5688888888888889, 0.47862867049936636, 0.47862867049936636, 0.23692688505618922, 0.23692688505618922]


In [13]:
# Преобразуем корни для отрезка [a, b]
for i in range(len(roots)):
    roots[i] = (b - a) / 2 * roots[i] + (a + b) / 2
print(roots)

[0.5, 0.7692346550528415, 0.2307653449471585, 0.9530899229693319, 0.04691007703066802]


In [18]:
# Квадратурная формула
sum = 0
for i in range(len(roots)):
    sum += A[i] * func(roots[i])

res_gauss = sum * (b - a) / 2
print(f"Значение интеграла: {res_gauss}")
print(f"Невязка с реальным решением: {abs(res_gauss) - abs(real_res)}")

Значение интеграла: -0.1978171088237481
Невязка с реальным решением: 2.8164761572968544e-07


In [25]:
# Остаток квадратурной формулы
# Нужна производная от функции степени 2 * n + 2. При n = 4: 10 степени.
# M_10 = 3.68369 * 10**9
# Берём max 10 производной на отрезке [a, b]
M_10 = 1.39157 * 10**7
n = 4
r_gauss = 2**(2 * n + 3) / ((2 * n + 3) * math.factorial(2 * n + 2)) \
          * (math.factorial(n + 1)**2 / math.factorial(2 * n + 2))**2 * M_10
r_gauss = r_gauss * ((b - a) / 2)**(2 * n + 3)
print(f"Оценка погрешности: {r_gauss}")

Оценка погрешности: 5.4896955256250204e-06
